In [3]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from dotenv import load_dotenv
from openai import OpenAI
import tiktoken

# eigene Module
from src.dataimport import list_files_with_extension_directory, load_text, list_files
from src.llmlib import num_tokens_from_string, generate_batch_input, split_jsonl_file, upload_batch_file, create_batch, check_batch_status, retrieve_and_save_batch_results

# Dateien laden

In [4]:
TXT_FILES_PATH = 'data/original/'
JSON_FILES_PATH = 'data/transformed/'

In [5]:
txt_files_directory_list = list_files_with_extension_directory(TXT_FILES_PATH, '.txt')
json_files_directory_list = list_files_with_extension_directory(JSON_FILES_PATH, '.json')

print(f"Anzahl Text-Dateien: {len(txt_files_directory_list)}")
print(f"Anzahl Brat-Dateien: {len(json_files_directory_list)}")

Anzahl Text-Dateien: 402
Anzahl Brat-Dateien: 402


In [6]:
# Dataframe mit Name und Inhalt der Text- und ann-Dateien erstellen 
df = pd.DataFrame()
df['txt_path'] = txt_files_directory_list
df['json_path'] = json_files_directory_list
df['txt_file'] = df['txt_path'].apply(lambda x: os.path.basename(x))
df['json_file'] = df['json_path'].apply(lambda x: os.path.basename(x))
df['txt'] = df['txt_path'].apply(load_text)
df['json'] = df['json_path'].apply(load_text)

print(df.shape)
df.head()

(402, 6)


,txt_path,json_path,txt_file,json_file,txt,json
0,data/original/essay001.txt,data/transformed/essay001.json,essay001.txt,essay001.json,Should students be taught to compete or to coo...,"{\n ""MajorClaims"": [\n {\n ""ID"": ""MC1..."
1,data/original/essay002.txt,data/transformed/essay002.json,essay002.txt,essay002.json,More people are migrating to other countries t...,"{\n ""MajorClaims"": [\n {\n ""ID"": ""MC1..."
2,data/original/essay003.txt,data/transformed/essay003.json,essay003.txt,essay003.json,International tourism is now more common than ...,"{\n ""MajorClaims"": [\n {\n ""ID"": ""MC1..."
3,data/original/essay004.txt,data/transformed/essay004.json,essay004.txt,essay004.json,International tourism is now more common than ...,"{\n ""MajorClaims"": [\n {\n ""ID"": ""MC1..."
4,data/original/essay005.txt,data/transformed/essay005.json,essay005.txt,essay005.json,Living and studying overseas\n\nIt is every st...,"{\n ""MajorClaims"": [\n {\n ""ID"": ""MC1..."


# Aufteilung in Trainings- und Testdatensatz

In [7]:
train_df, test_df = train_test_split(df, train_size=40, random_state=42)

# Formen der DataFrames
print(f"Training DataFrame: {train_df.shape}")
print(f"\nTest DataFrame: {test_df.shape}")

Training DataFrame: (40, 6)

Test DataFrame: (362, 6)


In [8]:
# Ausgabe der ersten Zeilen des Trainings-Datensatzes
train_df = train_df.sort_values(by='txt_file')
train_df.head()

,txt_path,json_path,txt_file,json_file,txt,json
20,data/original/essay021.txt,data/transformed/essay021.json,essay021.txt,essay021.json,Advertisements affects on consumer goods\n\nEv...,"{\n ""MajorClaims"": [\n {\n ""ID"": ""MC1..."
21,data/original/essay022.txt,data/transformed/essay022.json,essay022.txt,essay022.json,Young people should go to university or not\n\...,"{\n ""MajorClaims"": [\n {\n ""ID"": ""MC1..."
48,data/original/essay049.txt,data/transformed/essay049.json,essay049.txt,essay049.json,Do modern communication technologies benefit a...,"{\n ""MajorClaims"": [\n {\n ""ID"": ""MC1..."
50,data/original/essay051.txt,data/transformed/essay051.json,essay051.txt,essay051.json,Universities should give money to sport activi...,"{\n ""MajorClaims"": [\n {\n ""ID"": ""MC1..."
54,data/original/essay055.txt,data/transformed/essay055.json,essay055.txt,essay055.json,Should teenagers learn all school subjects/foc...,"{\n ""MajorClaims"": [\n {\n ""ID"": ""MC1..."


In [9]:
# Ausgabe der ersten Zeilen des Test-Datensatzes
test_df = test_df.sort_values(by='txt_file')
test_df.head()

,txt_path,json_path,txt_file,json_file,txt,json
0,data/original/essay001.txt,data/transformed/essay001.json,essay001.txt,essay001.json,Should students be taught to compete or to coo...,"{\n ""MajorClaims"": [\n {\n ""ID"": ""MC1..."
1,data/original/essay002.txt,data/transformed/essay002.json,essay002.txt,essay002.json,More people are migrating to other countries t...,"{\n ""MajorClaims"": [\n {\n ""ID"": ""MC1..."
2,data/original/essay003.txt,data/transformed/essay003.json,essay003.txt,essay003.json,International tourism is now more common than ...,"{\n ""MajorClaims"": [\n {\n ""ID"": ""MC1..."
3,data/original/essay004.txt,data/transformed/essay004.json,essay004.txt,essay004.json,International tourism is now more common than ...,"{\n ""MajorClaims"": [\n {\n ""ID"": ""MC1..."
4,data/original/essay005.txt,data/transformed/essay005.json,essay005.txt,essay005.json,Living and studying overseas\n\nIt is every st...,"{\n ""MajorClaims"": [\n {\n ""ID"": ""MC1..."


# Behandlung von Duplikaten
Es wurde fälschlicherweise angenommen, dass die Aufsätze keine Duplikate enthalten, da es sich um einen professionell erstellten Datensatz handelt. Es wurde, nachdem die Anfragen bereits an das LLM gesendet worden waren, jedoch festgestellt, dass ein Text dreimal und ein weiterer Text zweimal vorkommt. Die abweichende Annotation dieser Texte resultiert vermutlich aus der zuvor beschriebenen subjektiven Einschätzung der Annotatoren. Glücklicherweise befindet sich nur einer dieser Texte im Trainingsdatensatz, weshalb die Prompts und somit auch die Anfragen an das LLM nicht überarbeitet werden mussten. Die nachträgliche Behandlung der Duplikate spart Kosten und Zeit im Vergleich zu einer erneuten Übersendung der Anfragen an das LLM. 

Die Duplikate, welche sich im Testdatensatz befinden, werden bei der Evaluation des Modells entfernt (siehe 4. Notebook). Gemäß der Betrachtungen aus der EDA (1. Notebook) sind die Aufsätze mit der Nummer 171 und 210 aus dem Testdatensatz gleich zur Nummer 400 aus dem Trainingsdatensatz, weshalb sie beide aus dem Testdatensatz entfernt werden. Die Aufsätze mit der Nummer 209 und 377 sind gleich. Da beide Texte im Testdatensatz vorhanden sind, wird nur einer von beiden entfernt. Es sind folglich drei Aufsätze nachträglich zu entfernen. 3 von 402 Aufsätzen entsprechen ca. 0,75 % der Datenmenge.

In [8]:
duplicate_files = ['essay171.txt', 'essay209.txt', 'essay210.txt', 'essay377.txt', 'essay400.txt'] # Liste aus der EDA
# Überprüfung, welche der Duplikate in den Trainings- und Testdaten enthalten sind
print(f"Duplikate in Trainingsdaten: {train_df[train_df['txt_file'].isin(duplicate_files)]['txt_file'].tolist()}")
print(f"Duplikate in Testdaten: {test_df[test_df['txt_file'].isin(duplicate_files)]['txt_file'].tolist()}")

Duplikate in Trainingsdaten: ['essay400.txt']
Duplikate in Testdaten: ['essay171.txt', 'essay209.txt', 'essay210.txt', 'essay377.txt']


# Prompts erstellen

## Prompt-Bausteine

In [9]:
BUILDING_BLOCKS_PATH = 'prompts/building-blocks/'
PROMPTS_PATH = 'prompts/final-prompts/'

list_files(BUILDING_BLOCKS_PATH)

['chain-of-thought.txt',
 'output-structure.txt',
 'persona.txt',
 'task-description.txt']

In [10]:
# Zero-Shot Prompt
task_description = load_text(BUILDING_BLOCKS_PATH + 'task-description.txt')
persona = load_text(BUILDING_BLOCKS_PATH + 'persona.txt')
cot = load_text(BUILDING_BLOCKS_PATH + 'chain-of-thought.txt')
output_structure = load_text(BUILDING_BLOCKS_PATH + 'output-structure.txt')

## Zero Shot (ZS)

In [11]:
zs = task_description
zs_persona = persona + task_description
zs_cot = task_description + '\n' + cot
zs_persona_cot = persona + task_description + '\n' + cot

# Prompts als Textdateien speichern
with open(PROMPTS_PATH + 'zero-shot.txt', 'w') as f:
    f.write(zs)

with open(PROMPTS_PATH + 'zero-shot-persona.txt', 'w') as f:
    f.write(zs_persona)

with open(PROMPTS_PATH + 'zero-shot-cot.txt', 'w') as f:
    f.write(zs_cot)

with open(PROMPTS_PATH + 'zero-shot-persona-cot.txt', 'w') as f:
    f.write(zs_persona_cot)

## One-Shot (OS)

In [12]:
# 1 Beispiel aus dem Trainingsdatensatz auswählen für One-Shot-Prompt
examples_1 = train_df.sample(1, random_state=42)

# Zu dem Beispiel gehörige Text- und JSON-Daten extrahieren
os_txt = examples_1['txt'].values[0]
os_json = examples_1['json'].values[0]
os_example = f"## Input:\n{os_txt}\n## Output:\n{os_json}"

os = task_description + 'Here is one example of a text and its corresponding json data:\n' + os_example
os_persona = persona + task_description + '\n' + os_example
os_cot = task_description + '\n' + cot + '\n' + os_example
os_persona_cot = persona + task_description + '\n' + cot + '\n' + os_example

# Prompts als Textdateien speichern
with open(PROMPTS_PATH + 'one-shot.txt', 'w') as f:
    f.write(os)

with open(PROMPTS_PATH + 'one-shot-persona.txt', 'w') as f:
    f.write(os_persona)

with open(PROMPTS_PATH + 'one-shot-cot.txt', 'w') as f:
    f.write(os_cot)

with open(PROMPTS_PATH + 'one-shot-persona-cot.txt', 'w') as f:
    f.write(os_persona_cot)

## Few-Shot (FS)

In [13]:
# 10 Beispiel aus dem Trainingsdatensatz auswählen für Few-Shot-Prompt
examples_10 = train_df.sample(10, random_state=42)

few_shot_examples_10 = f"\nHere are 10 examples of text and their corresponding json data:\n" # Ergänzung einer Einführung für die 10 Beispiele
example_counter = 1
# Zu den Beispielen gehörige Text- und JSON-Daten extrahieren
for idx, row in examples_10.iterrows():
    example_str = f"\n# Example {example_counter}\n## Input:\n{row['txt']}\n## Output:\n{row['json']}"
    few_shot_examples_10 += example_str
    example_counter += 1

fs = task_description + few_shot_examples_10
fs_persona = persona + task_description + few_shot_examples_10
fs_cot = task_description + '\n' + cot + few_shot_examples_10
fs_persona_cot = persona + task_description + '\n' + cot + few_shot_examples_10

# Prompts als Textdateien speichern
with open(PROMPTS_PATH + 'few-shot-10.txt', 'w') as f:
    f.write(fs)

with open(PROMPTS_PATH + 'few-shot-10-persona.txt', 'w') as f:
    f.write(fs_persona)

with open(PROMPTS_PATH + 'few-shot-10-cot.txt', 'w') as f:
    f.write(fs_cot)

with open(PROMPTS_PATH + 'few-shot-10-persona-cot.txt', 'w') as f:
    f.write(fs_persona_cot)

In [14]:
# 20 Beispiel aus dem Trainingsdatensatz auswählen für Few-Shot-Prompt
examples_20 = train_df.sample(20, random_state=42)

few_shot_examples_20 = f"\nHere are 20 examples of text and their corresponding json data:\n" # Ergänzung einer Einführung für die 20 Beispiele
example_counter = 1
# Zu den Beispielen gehörige Text- und JSON-Daten extrahieren
for idx, row in examples_20.iterrows():
    example_str = f"\n# Example {example_counter}\n## Input:\n{row['txt']}\n## Output:\n{row['json']}"
    few_shot_examples_20 += example_str
    example_counter += 1

fs = task_description + few_shot_examples_20
fs_persona = persona + task_description + few_shot_examples_20
fs_cot = task_description + '\n' + cot + few_shot_examples_20
fs_persona_cot = persona + task_description + '\n' + cot + few_shot_examples_20

# Prompts als Textdateien speichern
with open(PROMPTS_PATH + 'few-shot-20.txt', 'w') as f:
    f.write(fs)

with open(PROMPTS_PATH + 'few-shot-20-persona.txt', 'w') as f:
    f.write(fs_persona)

with open(PROMPTS_PATH + 'few-shot-20-cot.txt', 'w') as f:
    f.write(fs_cot)

with open(PROMPTS_PATH + 'few-shot-20-persona-cot.txt', 'w') as f:
    f.write(fs_persona_cot)

In [15]:
# 40 Beispiel aus dem Trainingsdatensatz auswählen für Few-Shot-Prompt
examples_40 = train_df.sample(40, random_state=42)

few_shot_str_40 = f"\nHere are 40 examples of text and their corresponding json data:\n" # Ergänzung einer Einführung für die 20 Beispiele
example_counter = 1
# Zu den Beispielen gehörige Text- und JSON-Daten extrahieren
for idx, row in examples_40.iterrows():
    example_str = f"\n# Example {example_counter}\n## Input:\n{row['txt']}\n## Output:\n{row['json']}"
    few_shot_str_40 += example_str
    example_counter += 1

fs = task_description + few_shot_str_40
fs_persona = persona + task_description + few_shot_str_40
fs_cot = task_description + '\n' + cot + few_shot_str_40
fs_persona_cot = persona + task_description + '\n' + cot + few_shot_str_40

# Prompts als Textdateien speichern
with open(PROMPTS_PATH + 'few-shot-40.txt', 'w') as f:
    f.write(fs)

with open(PROMPTS_PATH + 'few-shot-40-persona.txt', 'w') as f:
    f.write(fs_persona)

with open(PROMPTS_PATH + 'few-shot-40-cot.txt', 'w') as f:
    f.write(fs_cot)

with open(PROMPTS_PATH + 'few-shot-40-persona-cot.txt', 'w') as f:
    f.write(fs_persona_cot)

## Promptvariationen

In [16]:
# Eigentlich wird die os-Bibliothek bereits obengeladen. Da es aber vereinzelt zu Fehlermeldungen kam, wird sie hier nochmals geladen.
import os

prompt_files_directory_list = list_files_with_extension_directory(PROMPTS_PATH, '.txt')
prompt_files_directory_list
prompt_files_list = [os.path.basename(x) for x in prompt_files_directory_list]
# Entfernen der txt-Endung
prompt_names = [x.split('.')[0] for x in prompt_files_list]

# Vorbereiten des DataFrames mit den Prompt-Variationen
prompt_df = pd.DataFrame()
prompt_df['prompt_name'] = prompt_names
prompt_df['prompt_txt'] = prompt_files_directory_list
prompt_df['prompt_txt'] = prompt_df['prompt_txt'].apply(load_text)
print(F"Es gibt {prompt_df.shape[0]} Prompts")

Es gibt 20 Prompts


# Berechnung der Tokenanzahl

In [17]:
model = 'gpt-4o-mini'

# Beispiel zur Nachvollziehbarkeit der Tokenisierung
encoding = tiktoken.encoding_for_model(model)
print(encoding)

sample_txt = "This is a sample sentence."
# Text encodieren
token_integer = encoding.encode(sample_txt) # mit .decode() kann der Text wieder dekodiert werden. 
token_bytes = [encoding.decode_single_token_bytes(token) for token in token_integer] # Integer Token können wiederum in Bytes umgewandelt werden, die sie repräsentieren.
print(f"Beispieltext: {sample_txt}")
print(f"Encodierter Text (Integer): {token_integer}")
print(f"Encodierter Text (Bytes): {token_bytes}")


count_tokens = num_tokens_from_string(sample_txt, model)
print(f"Anzahl Tokens: {count_tokens}")

# Codebausteine zur Berechnung der Tokenanzahl und Encoding entnommen aus: https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken

<Encoding 'o200k_base'>
Beispieltext: This is a sample sentence.
Encodierter Text (Integer): [2500, 382, 261, 10176, 21872, 13]
Encodierter Text (Bytes): [b'This', b' is', b' a', b' sample', b' sentence', b'.']
Anzahl Tokens: 6


In [18]:
# Dataframe mit Tokenanzahl und Anzahl der Batches für die Verarbeitung des Testdatensatzes
prompt_df['token_count'] = prompt_df['prompt_txt'].apply(num_tokens_from_string, model_name=model)
prompt_df = prompt_df.sort_values(by='token_count')
# Schätzung für die Anzahl der Batches, die für die Verarbeitung des Testdatensatzes benötigt werden
# Die Tokens für den User-Input sind hier nicht berücksichtigt. Der reale Wert wird daher höher sein.
prompt_df['max_lines_jsonl'] = round(20_000_000 / prompt_df['token_count'], 0) # 20 Mio. entspricht der max. Anzahl an Tokens pro JSONL-Datei (enqueued tokens) für den verwendeten Client
prompt_df['#batches'] = round(test_df.shape[0] / prompt_df['max_lines_jsonl'], 2) # Anzahl der Batches, die für die Verarbeitung benötigt werden
prompt_df

,prompt_name,prompt_txt,token_count,max_lines_jsonl,#batches
19,zero-shot,You will be given a text. Extract the argument...,82,243902.0,0.00
18,zero-shot-persona,You are a expert in Argument Mining and theref...,105,190476.0,0.00
16,zero-shot-cot,You will be given a text. Extract the argument...,480,41667.0,0.01
17,zero-shot-persona-cot,You are a expert in Argument Mining and theref...,503,39761.0,0.01
15,one-shot,You will be given a text. Extract the argument...,1780,11236.0,0.03
14,one-shot-persona,You are a expert in Argument Mining and theref...,1790,11173.0,0.03
12,one-shot-cot,You will be given a text. Extract the argument...,2166,9234.0,0.04
13,one-shot-persona-cot,You are a expert in Argument Mining and theref...,2189,9137.0,0.04
3,few-shot-10,You will be given a text. Extract the argument...,13848,1444.0,0.25
2,few-shot-10-persona,You are a expert in Argument Mining and theref...,13871,1442.0,0.25


# Schätzungen der Anfragen und Kosten

## Schätzungen der Anfragen bzw. Batches
**Hinweis**

Die Anfragen können nicht in einem gemeinsamen Batch auf einmal übermittelt werden, da es ansonsten zu einem Fehler kommt. Für die API von OpenAI bestehen Anfragebegrenzungen, die es bei der Erstellung der Batches zu berücksichtigen gilt. Sofern die Anfragebegrenzungen überschritten wurden, kann es beispielsweise zu folgender Fehlermeldung kommen:
```
Enqueued token limit reached for gpt-4o-mini in organization. Limit: 20,000,000 enqueued tokens. Please try again once some in_progress batches have been completed.
```
Nach eigener Erfahrung tritt solch eine Fehlermeldung unter zwei Bedingungen auf. Zum einen, wenn die Anfragebegrenzung überschritten wurde, und zum anderen, wenn die Anfragen zu schnell hintereinander gestellt werden. Es wird deshalb empfohlen, die Anfragen nacheinander in Auftrag zu geben, sobald der vorherige Batch abgeschlossen ist. Sofern eine Batch-Datei hochgeladen wurde und der Batch an sich jedoch fehlschlägt, kann die Batch-Datei weiterhin anhand ihrer ID verwendet werden. Die Funktion check_batch_status kann beliebig oft ohne zusätzliche Kosten verwendet werden, um den Status eines Batches zu überprüfen.

In [19]:
# Unterteilung der Prompts anhand der Anzahl der enthaltenen Beispiele
zs_prompt_df = prompt_df[prompt_df['prompt_name'].str.contains('zero-shot')]
os_prompt_df = prompt_df[prompt_df['prompt_name'].str.contains('one-shot')]
fs10_prompt_df = prompt_df[prompt_df['prompt_name'].str.contains('few-shot-10')]
fs20_prompt_df = prompt_df[prompt_df['prompt_name'].str.contains('few-shot-20')]
fs40_prompt_df = prompt_df[prompt_df['prompt_name'].str.contains('few-shot-40')]

# Anzahl der Zeilen für die verschiedenen Prompts
zs_rows = zs_prompt_df.shape[0]
os_rows = os_prompt_df.shape[0]
fs10_rows = fs10_prompt_df.shape[0]
fs20_rows = fs20_prompt_df.shape[0]
fs40_rows = fs40_prompt_df.shape[0]

# Gesamtanzahl der Kombinationen für die Verarbeitung des Testdatensatzes bestimmen
combinations = test_df.shape[0] * (zs_rows + os_rows + fs10_rows + fs20_rows + fs40_rows)
print(f"Es gibt insgesamt {combinations} Kombinationen, die verarbeitet werden müssen.")
print(f"Davon entfallen {test_df.shape[0] * zs_rows} auf Zero-Shot-Prompts.")
print(f"Davon entfallen {test_df.shape[0] * os_rows} auf One-Shot-Prompts.")
print(f"Davon entfallen {test_df.shape[0] * fs10_rows} auf Few-Shot-Prompts mit 10 Beispielen.")
print(f"Davon entfallen {test_df.shape[0] * fs20_rows} auf Few-Shot-Prompts mit 20 Beispielen.")
print(f"Davon entfallen {test_df.shape[0] * fs40_rows} auf Few-Shot-Prompts mit 40 Beispielen.")

Es gibt insgesamt 7240 Kombinationen, die verarbeitet werden müssen.
Davon entfallen 1448 auf Zero-Shot-Prompts.
Davon entfallen 1448 auf One-Shot-Prompts.
Davon entfallen 1448 auf Few-Shot-Prompts mit 10 Beispielen.
Davon entfallen 1448 auf Few-Shot-Prompts mit 20 Beispielen.
Davon entfallen 1448 auf Few-Shot-Prompts mit 40 Beispielen.


In [20]:
# Berechnung des Tokenverbrauchs pro Batch. Das limit des verwendeten Clients liegt bei 2,000,000 enqueued tokens
zs_prompt_df_sum = zs_prompt_df['token_count'].sum()
os_prompt_df_sum = os_prompt_df['token_count'].sum()
fs10_df_sum = fs10_prompt_df['token_count'].sum()
fs20_df_sum = fs20_prompt_df['token_count'].sum()
fs40_df_sum = fs40_prompt_df['token_count'].sum()
#requests = 1448 # Anzahl der Anfragen, die in einem Batch verarbeitet werden

# Anzahl der Batch-Anfragen
zs_batches = round(zs_prompt_df['#batches'].sum(), 2)
os_batches = round(os_prompt_df['#batches'].sum(), 2)
fs10_batches = round(fs10_prompt_df['#batches'].sum(), 2)
fs20_batches = round(fs20_prompt_df['#batches'].sum(), 2)
fs40_batches = round(fs40_prompt_df['#batches'].sum(), 2)

print("Schätzung der Anzahl der Batches für die Verarbeitung des Testdatensatzes:")
print(f"Zero-Shot-Prompts: {zs_batches} Batches mit {zs_prompt_df_sum} Tokens")
print(f"One-Shot-Prompts: {os_batches} Batches mit {os_prompt_df_sum} Tokens")
print(f"Few-Shot-Prompts mit 10 Beispielen: {fs10_batches} Batches mit {fs10_df_sum} Tokens")
print(f"Few-Shot-Prompts mit 20 Beispielen: {fs20_batches} Batches mit {fs20_df_sum} Tokens")
print(f"Few-Shot-Prompts mit 40 Beispielen: {fs40_batches} Batches mit {fs40_df_sum} Tokens")
print(f"Es gibt insgesamt {(zs_batches + os_batches + fs10_batches + fs20_batches + fs40_batches):.2f} Batches")

Schätzung der Anzahl der Batches für die Verarbeitung des Testdatensatzes:
Zero-Shot-Prompts: 0.02 Batches mit 1170 Tokens
One-Shot-Prompts: 0.14 Batches mit 7925 Tokens
Few-Shot-Prompts mit 10 Beispielen: 1.02 Batches mit 56236 Tokens
Few-Shot-Prompts mit 20 Beispielen: 2.02 Batches mit 111568 Tokens
Few-Shot-Prompts mit 40 Beispielen: 3.94 Batches mit 217036 Tokens
Es gibt insgesamt 7.14 Batches


Hier wird deutlich, dass die Unterteilung der Batches anhand der Anzahl an Beispielen im Prompt aufgrund der unterschiedlichen Komplexität und damit Tokenanzahl nicht sinnvoll ist. So könnten die ZS- und OS-Prompts in einem Batch zusammengefasst werden.

## Schätzung der anfallenden Kosten

In [21]:
# Schätzung der anfallenden Kosten anhand der Tokenanzahl
print(f"{"-" * 60}") # Trennlinie
print("Schätzung der Kosten für die Verarbeitung des Testdatensatzes")
print(f"{"-" * 60}")
prompt_token_sum = prompt_df['token_count'].sum()
print(f"Input-Token:\nDie Summe der Input-Tokenanzahl aller Prompts beträgt: {prompt_token_sum:,} Tokens")
test_token_sum = prompt_token_sum * test_df.shape[0]
print(f"Multipliziert mit der Anzahl der Testdurchläufe ergibt das: {test_token_sum:,} Tokens für Input-Token")
max_enqueued_tokens = 20_000_000 # Maximale Anzahl an Tokens, die von der Batch API
print(f"Das entspicht bei einer maximalen Anzahl von {max_enqueued_tokens:,} Tokens pro JSONL-Datei: {test_token_sum/20_000_000:.2f} Batches") 
input_token_price = 0.15 # input token price per 1 Mio tokens
output_token_price = 0.6 # output token price per 1 Mio tokens
input_token_cost = input_token_price * test_token_sum/1_000_000
print(f"Die Kosten für die Input-Tokens betragen: {input_token_cost:.2f} $") 
min_output_token_count = 450 # Minimalwert für Tokenanzahl für ann-Dateien aus EDA.
max_output_token_count = 1_620 # aufgerundeter Maximalwert für Tokenanzahl für ann-Dateien aus EDA. Umfang der Ausgabe des LLMs kann auch außerhalb des Bereichs liegen. 
min_output_token_sum = min_output_token_count * test_df.shape[0]
max_output_token_sum = max_output_token_count * test_df.shape[0]
print(f"\nOutput-Token:\nDie Output-Tokenanzahl multipliziert mit der Anzahl der Aufsätze im Testdatensaatz beträgt zwischen: {min_output_token_sum:,} und {max_output_token_sum:,} Tokens")
min_output_token_cost = output_token_price * min_output_token_sum/1_000_000
max_output_token_cost = output_token_price * max_output_token_sum/1_000_000
print(f"Die Kosten für die Output-Tokens betragen zwischen: {min_output_token_cost:.2f} $ und {max_output_token_cost:.2f} $")
total_cost_min = input_token_cost + min_output_token_cost
total_cost_max = input_token_cost + max_output_token_cost
print(f"\nGesamt:\nDie Gesamtkosten liegen schätzungsweise in einem Bereich von {total_cost_min:.2f} $ und {total_cost_max:.2f} $")
print(f"Bei der Anwendung der Batch-API gibt es einen Rabatt von 50% auf die Tokenpreise. Damit würden die Kosten zwischen {total_cost_min/2:.2f} $ und {total_cost_max/2:.2f} $ liegen.")

# Quelle für Tokenpreise: https://openai.com/api/pricing/ (Stand 01/2025) 

------------------------------------------------------------
Schätzung der Kosten für die Verarbeitung des Testdatensatzes
------------------------------------------------------------
Input-Token:
Die Summe der Input-Tokenanzahl aller Prompts beträgt: 393,935 Tokens
Multipliziert mit der Anzahl der Testdurchläufe ergibt das: 142,604,470 Tokens für Input-Token
Das entspicht bei einer maximalen Anzahl von 20,000,000 Tokens pro JSONL-Datei: 7.13 Batches
Die Kosten für die Input-Tokens betragen: 21.39 $

Output-Token:
Die Output-Tokenanzahl multipliziert mit der Anzahl der Aufsätze im Testdatensaatz beträgt zwischen: 162,900 und 586,440 Tokens
Die Kosten für die Output-Tokens betragen zwischen: 0.10 $ und 0.35 $

Gesamt:
Die Gesamtkosten liegen schätzungsweise in einem Bereich von 21.49 $ und 21.74 $
Bei der Anwendung der Batch-API gibt es einen Rabatt von 50% auf die Tokenpreise. Damit würden die Kosten zwischen 10.74 $ und 10.87 $ liegen.


# LLM Abfrage

In [22]:
# API-Key aus .env-Datei laden  
load_dotenv()
openai_api = os.getenv("OPENAI_API_KEY")
                       
client = OpenAI(api_key=openai_api)

## strukturiertes Ausgabeformat

In [23]:
response_format = {
    "type": "object",
    "properties": {
        "MajorClaims": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "ID": {
                        "type": "string"
                    },
                    "Text": {
                        "type": "string"
                    }
                },
                "required": ["ID", "Text"],
                "additionalProperties": False
            }
        },
        "Claims": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "ID": {
                        "type": "string"
                    },
                    "Text": {
                        "type": "string"
                    }
                },
                "required": ["ID", "Text"],
                "additionalProperties": False
            }
        },
        "Premises": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "ID": {
                        "type": "string"
                    },
                    "Text": {
                        "type": "string"
                    }
                },
                "required": ["ID", "Text"],
                "additionalProperties": False
            }
        },
        "ArgumentativeRelations": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "Origin": {
                        "type": "string"
                    },
                    "Relation": {
                        "type": "string",
                        "enum": ["for", "against", "supports", "attacks"]
                    },
                    "Target": {
                        "type": "string"
                    }
                },
                "required": ["Origin", "Relation", "Target"],
                "additionalProperties": False
            }
        }
    },
    "required": ["MajorClaims", "Claims", "Premises", "ArgumentativeRelations"],
    "additionalProperties": False
    }

# Der Ansatz unter Verwendung von Pydantic hat nicht funktioniert und wurde daher verworfen.

# herangezogene Quellen zur Erstellung des strukturierten Ausgabeformats:
# - https://platform.openai.com/docs/guides/structured-outputs
# - https://cookbook.openai.com/examples/structured_outputs_intro
# - https://python.langchain.com/docs/concepts/structured_outputs/

## Batch API Input Dateien erstellen (JSONL)

In [24]:
BATCH_INPUT_PATH = "batch_api/input/"
BATCH_OUTPUT_PATH = "batch_api/output/"

In [25]:
# Batch API Input-Datei erstellen (JSONL-Format)
full_jsonl_path = generate_batch_input(test_df, prompt_df, "full_batch_input", response_format, BATCH_INPUT_PATH)

# Aufteilung der zuvor erstellten JSONL-Datei in mehrere Dateien zur schrittweisen Verarbeitung durch die Batch-API
# Als Kriterium zum Aufteilen wird die maximale Anzahl an Tokens pro JSONL-Datei (20 Mio.) verwendet.
split_jsonl_file(full_jsonl_path)
print(f"Die Datei wurde in mehrere Dateien aufgeteilt, welche unter folgendem Pfad gespeichert sind: {BATCH_INPUT_PATH}")

Die Datei wurde in mehrere Dateien aufgeteilt, welche unter folgendem Pfad gespeichert sind: batch_api/input/


In [26]:
batch_input_files_list = list_files(BATCH_INPUT_PATH)
batch_input_files_list

['batch_input_1.jsonl',
 'batch_input_2.jsonl',
 'batch_input_3.jsonl',
 'batch_input_4.jsonl',
 'batch_input_5.jsonl',
 'batch_input_6.jsonl',
 'batch_input_7.jsonl',
 'batch_input_8.jsonl',
 'batch_input_9.jsonl']

In [41]:
# Anzahl der Zeilen in den JSONL-Dateien. Eine Zeile entspricht einer Anfrage an die Batch-API
line_counts = [sum(1 for line in open(BATCH_INPUT_PATH + file)) for file in batch_input_files_list]
print(f"Liste der Anfragen pro JSONL-Datei: {line_counts}")
print(f"Summe der Anfragen: {sum(line_counts)}")

Liste der Anfragen pro JSONL-Datei: [3816, 878, 622, 549, 323, 322, 320, 320, 90]
Summe der Anfragen: 7240


Je komplexer die Prompts, desto mehr Token werden benötigt. Folglich besteht ein Batch mit komplexen Prompts aus weniger Anfragen als ein Batch mit einfachen Prompts. 

## Batches hochladen, erstellen und abfragen

Beschreibung der Status-Codes:

| Status       | Description                                                                 |
|--------------|-----------------------------------------------------------------------------|
| validating   | the input file is being validated before the batch can begin                |
| failed       | the input file has failed the validation process                            |
| in_progress  | the input file was successfully validated and the batch is currently being run |
| finalizing   | the batch has completed and the results are being prepared                  |
| completed    | the batch has been completed and the results are ready                      |
| expired      | the batch was not able to be completed within the 24-hour time window       |
| cancelling   | the batch is being cancelled (may take up to 10 minutes)                    |
| cancelled    | the batch was cancelled                                                     |

Tabelle entnommen aus: https://platform.openai.com/docs/guides/batch/batch-api

### Batch 1

In [ ]:
# Upload der Batch Input-Datei auf die OpenAI-Plattform
batch_file_1 = upload_batch_file(BATCH_INPUT_PATH + "batch_input_1.jsonl", client)
batch_file_1

FileObject(id='file-73X2eNMUnxhwLndGXvNdZ5', bytes=84798929, created_at=1736407855, filename='batch_input_1.jsonl', object='file', purpose='batch', status='processed', status_details=None)

In [ ]:
# Erstellen eines Batches
metadata_dict = {"description": "Batch 1/9 for Argument Mining"}
batch_1 = create_batch(batch_file_1.id, metadata_dict, client) # sofern die Batch Datei bereits hochgeladen wurde, aber nicht erfolgreich war, kann die Batch-ID erneut verwendet werden. Ein erneuter Uplaod ist nicht notwendig und würde der Datei eine neue ID zuweisen.
print(batch_1)

Batch(id='batch_677f7b3b1b48819086a5a27b7173c2c3', completion_window='24h', created_at=1736407867, endpoint='/v1/chat/completions', input_file_id='file-73X2eNMUnxhwLndGXvNdZ5', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736494267, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'Batch 1/9 for Argument Mining'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


In [ ]:
# Status des Batches abfragen - Diese Funktion kann mehrfach aufgerufen werden, um den Status des Batches zu überprüfen, ohne Zusatzkosten zu verursachen.
# Die Bearbeitung des Batches kann bis zu 24 Stunden dauern, funktioniert aber in der Regel schneller.
batch_1_status = check_batch_status(batch_1.id, client)
print(batch_1_status)

Status: completed

Beschreibung des Batches: Batch 1/9 for Argument Mining
Anfragen gesamt: 3816
Davon erfolgreich: 3816
Davon fehlerhaft: 0
Erfolgreiche Abfragen können abgerufen werden mit ID: file-Bwq5PPm4rnvL2jk7KFt2cL
Keine fehlerhaften Abfragen zum herunterladen vorhanden.

Batch(id='batch_677f7b3b1b48819086a5a27b7173c2c3', completion_window='24h', created_at=1736407867, endpoint='/v1/chat/completions', input_file_id='file-73X2eNMUnxhwLndGXvNdZ5', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1736411765, error_file_id=None, errors=None, expired_at=None, expires_at=1736494267, failed_at=None, finalizing_at=1736411356, in_progress_at=1736407871, metadata={'description': 'Batch 1/9 for Argument Mining'}, output_file_id='file-Bwq5PPm4rnvL2jk7KFt2cL', request_counts=BatchRequestCounts(completed=3816, failed=0, total=3816))


In [ ]:
# Abrufen und Speichern der Batch-Ergebnisse (Output-Datei)
batch_1_results = retrieve_and_save_batch_results(batch_1_status.output_file_id, BATCH_OUTPUT_PATH, "output-batch-1", client)
print(batch_1_results[:1000])

{"id": "batch_req_677f88dcb8c881908e9275650fab34fa", "custom_id": "zero-shot_essay001.txt", "response": {"status_code": 200, "request_id": "82330270e662bb47722cf42d0c0ab869", "body": {"id": "chatcmpl-AnhIbjUujngIZN41Ogf87fCAje0x7", "object": "chat.completion", "created": 1736407877, "model": "gpt-4o-mini-2024-07-18", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\"MajorClaims\":[{\"ID\":\"MC1\",\"Text\":\"We should attach more importance to cooperation during primary education.\"}],\"Claims\":[{\"ID\":\"C1\",\"Text\":\"Competition can effectively promote the development of economy.\"},{\"ID\":\"C2\",\"Text\":\"Cooperation helps children learn interpersonal skills.\"},{\"ID\":\"C3\",\"Text\":\"Competition makes society more effective.\"},{\"ID\":\"C4\",\"Text\":\"Victory in competition often requires cooperation.\"}],\"Premises\":[{\"ID\":\"P1\",\"Text\":\"Companies improve their products and services to survive in competition.\"},{\"ID\":\"P2\",\"Text\":\"Inter

### Batch 2

In [ ]:
# Upload der Batch Input-Datei auf die OpenAI-Plattform
batch_file_2 = upload_batch_file(BATCH_INPUT_PATH + "batch_input_2.jsonl", client)
batch_file_2

FileObject(id='file-NyvUx7xYfyFu6C4QNZ6qYs', bytes=80171245, created_at=1736498896, filename='batch_input_2.jsonl', object='file', purpose='batch', status='processed', status_details=None)

In [ ]:
# Erstellen eines Batches
metadata_dict = {"description": "Batch 2/9 for Argument Mining"}
batch_2 = create_batch(batch_file_2.id, metadata_dict, client) # sofern die Batch Datei bereits hochgeladen wurde, aber nicht erfolgreich war, kann die Batch-ID erneut verwendet werden. Ein erneuter Uplaod ist nicht notwendig und würde der Datei eine neue ID zuweisen.
print(batch_2)

Batch(id='batch_6780ded3cd58819080b492f6647492b4', completion_window='24h', created_at=1736498899, endpoint='/v1/chat/completions', input_file_id='file-NyvUx7xYfyFu6C4QNZ6qYs', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736585299, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'Batch 2/9 for Argument Mining'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


In [ ]:
# Status des Batches abfragen - Diese Funktion kann mehrfach aufgerufen werden, um den Status des Batches zu überprüfen, ohne Zusatzkosten zu verursachen.
batch_2_status = check_batch_status(batch_2.id, client)
print(batch_2_status)

Status: completed

Beschreibung des Batches: Batch 2/9 for Argument Mining
Anfragen gesamt: 878
Davon erfolgreich: 878
Davon fehlerhaft: 0
Erfolgreiche Abfragen können abgerufen werden mit ID: file-Fg7gsDHnWPhPKX5BLzQhaH
Keine fehlerhaften Abfragen zum herunterladen vorhanden.

Batch(id='batch_6780ded3cd58819080b492f6647492b4', completion_window='24h', created_at=1736498899, endpoint='/v1/chat/completions', input_file_id='file-NyvUx7xYfyFu6C4QNZ6qYs', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1736499712, error_file_id=None, errors=None, expired_at=None, expires_at=1736585299, failed_at=None, finalizing_at=1736499609, in_progress_at=1736498902, metadata={'description': 'Batch 2/9 for Argument Mining'}, output_file_id='file-Fg7gsDHnWPhPKX5BLzQhaH', request_counts=BatchRequestCounts(completed=878, failed=0, total=878))


In [ ]:
# Abrufen und Speichern der Batch-Ergebnisse (Output-Datei)
batch_2_results = retrieve_and_save_batch_results(batch_2_status.output_file_id, BATCH_OUTPUT_PATH, "output-batch-2", client)
print(batch_2_results[:1000])

{"id": "batch_req_6780e1bbdd408190b622fa829f21b366", "custom_id": "few-shot-10-cot_essay220.txt", "response": {"status_code": 200, "request_id": "7efa009d49bf99540b29b3fb45a5b4d6", "body": {"id": "chatcmpl-Ao4yrOhe57gU4dz9cROmIgRKN2NNL", "object": "chat.completion", "created": 1736498909, "model": "gpt-4o-mini-2024-07-18", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\n  \"MajorClaims\": [\n    {\n      \"ID\": \"MC1\",\n      \"Text\": \"Learning to be independent is essential for young adults\"\n    },\n    {\n      \"ID\": \"MC2\",\n      \"Text\": \"staying longer with parents is a better choice\"\n    }\n  ],\n  \"Claims\": [\n    {\n      \"ID\": \"C1\",\n      \"Text\": \"staying with the parents for longer time does more benefits than disadvantages to the young adult\"\n    },\n    {\n      \"ID\": \"C2\",\n      \"Text\": \"the young adult can have more experience with his parents\"\n    },\n    {\n      \"ID\": \"C3\",\n      \"Text\": \"living at ho

### Batch 3

In [ ]:
# Upload der Batch Input-Datei auf die OpenAI-Plattform
batch_file_3 = upload_batch_file(BATCH_INPUT_PATH + "batch_input_3.jsonl", client)
batch_file_3

FileObject(id='file-1VeM9QWs9utTGbREGjvauV', bytes=79782274, created_at=1736585156, filename='batch_input_3.jsonl', object='file', purpose='batch', status='processed', status_details=None)

In [ ]:
# Erstellen eines Batches
metadata_dict = {"description": "Batch 3/9 for Argument Mining"}
batch_3 = create_batch(batch_file_3.id, metadata_dict, client) # sofern die Batch Datei bereits hochgeladen wurde, aber nicht erfolgreich war, kann die Batch-ID erneut verwendet werden. Ein erneuter Uplaod ist nicht notwendig und würde der Datei eine neue ID zuweisen.
print(batch_3)

Batch(id='batch_67822fc8058c8190b864f42b92ad646d', completion_window='24h', created_at=1736585160, endpoint='/v1/chat/completions', input_file_id='file-1VeM9QWs9utTGbREGjvauV', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736671560, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'Batch 3/9 for Argument Mining'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


In [ ]:
# Status des Batches abfragen - Diese Funktion kann mehrfach aufgerufen werden, um den Status des Batches zu überprüfen, ohne Zusatzkosten zu verursachen.
batch_3_status = check_batch_status(batch_3.id, client)
print(batch_3_status)

Status: completed

Beschreibung des Batches: Batch 3/9 for Argument Mining
Anfragen gesamt: 622
Davon erfolgreich: 622
Davon fehlerhaft: 0
Erfolgreiche Abfragen können abgerufen werden mit ID: file-Dn8EPeVM9BjT1VpRaaAnvS
Keine fehlerhaften Abfragen zum herunterladen vorhanden.

Batch(id='batch_67822fc8058c8190b864f42b92ad646d', completion_window='24h', created_at=1736585160, endpoint='/v1/chat/completions', input_file_id='file-1VeM9QWs9utTGbREGjvauV', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1736586694, error_file_id=None, errors=None, expired_at=None, expires_at=1736671560, failed_at=None, finalizing_at=1736586509, in_progress_at=1736585162, metadata={'description': 'Batch 3/9 for Argument Mining'}, output_file_id='file-Dn8EPeVM9BjT1VpRaaAnvS', request_counts=BatchRequestCounts(completed=622, failed=0, total=622))


In [ ]:
# Abrufen und Speichern der Batch-Ergebnisse (Output-Datei)
batch_3_results = retrieve_and_save_batch_results(batch_3_status.output_file_id, BATCH_OUTPUT_PATH, "output-batch-3", client)
print(batch_3_results[:1000])

{"id": "batch_req_6782350f50148190ae1ecbe23a044ce3", "custom_id": "few-shot-20_essay389.txt", "response": {"status_code": 200, "request_id": "429b27845229d782546ed0cbc4fb17c3", "body": {"id": "chatcmpl-AoRVz57qqhjznstY4shTiOqFnQRQC", "object": "chat.completion", "created": 1736585531, "model": "gpt-4o-mini-2024-07-18", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\n  \"MajorClaims\": [\n    {\n      \"ID\": \"MC1\",\n      \"Text\": \"education plays an important role in the socioeconomic status of a country\"\n    },\n    {\n      \"ID\": \"MC2\",\n      \"Text\": \"education is the single most important factor in the development of a country\"\n    }\n  ],\n  \"Claims\": [\n    {\n      \"ID\": \"C1\",\n      \"Text\": \"education is undeniably an economic necessity\"\n    },\n    {\n      \"ID\": \"C2\",\n      \"Text\": \"not many can afford to send their children to school in a developing country\"\n    },\n    {\n      \"ID\": \"C3\",\n      \"Text\": \"

### Batch 4

In [ ]:
# Upload der Batch Input-Datei auf die OpenAI-Plattform
batch_file_4 = upload_batch_file(BATCH_INPUT_PATH + "batch_input_4.jsonl", client)
batch_file_4

FileObject(id='file-CUxcAUKiZJZ64KkV8YAsG4', bytes=79467621, created_at=1736670809, filename='batch_input_4.jsonl', object='file', purpose='batch', status='processed', status_details=None)

In [ ]:
# Erstellen eines Batches
metadata_dict = {"description": "Batch 4/9 for Argument Mining"}
batch_4 = create_batch(batch_file_4.id, metadata_dict, client) # sofern die Batch Datei bereits hochgeladen wurde, aber nicht erfolgreich war, kann die Batch-ID erneut verwendet werden. Ein erneuter Uplaod ist nicht notwendig und würde der Datei eine neue ID zuweisen.
print(batch_4)

Batch(id='batch_67837fec4f04819088f396053ef790f2', completion_window='24h', created_at=1736671212, endpoint='/v1/chat/completions', input_file_id='file-CUxcAUKiZJZ64KkV8YAsG4', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736757612, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'Batch 4/9 for Argument Mining'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


In [ ]:
# Status des Batches abfragen - Diese Funktion kann mehrfach aufgerufen werden, um den Status des Batches zu überprüfen, ohne Zusatzkosten zu verursachen.
batch_4_status = check_batch_status(batch_4.id, client)
print(batch_4_status)

Status: completed

Beschreibung des Batches: Batch 4/9 for Argument Mining
Anfragen gesamt: 549
Davon erfolgreich: 549
Davon fehlerhaft: 0
Erfolgreiche Abfragen können abgerufen werden mit ID: file-CTPtvsw1e79hupNn1R1DXf
Keine fehlerhaften Abfragen zum herunterladen vorhanden.

Batch(id='batch_67837fec4f04819088f396053ef790f2', completion_window='24h', created_at=1736671212, endpoint='/v1/chat/completions', input_file_id='file-CUxcAUKiZJZ64KkV8YAsG4', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1736673370, error_file_id=None, errors=None, expired_at=None, expires_at=1736757612, failed_at=None, finalizing_at=1736673312, in_progress_at=1736671215, metadata={'description': 'Batch 4/9 for Argument Mining'}, output_file_id='file-CTPtvsw1e79hupNn1R1DXf', request_counts=BatchRequestCounts(completed=549, failed=0, total=549))


In [ ]:
# Abrufen und Speichern der Batch-Ergebnisse (Output-Datei)
batch_4_results = retrieve_and_save_batch_results(batch_4_status.output_file_id, BATCH_OUTPUT_PATH, "output-batch-4", client)
print(batch_4_results[:1000])

{"id": "batch_req_67838820cb908190bb1db05670129112", "custom_id": "few-shot-20-cot_essay276.txt", "response": {"status_code": 200, "request_id": "856774c31c28b8df1dfb7b6a6a981df5", "body": {"id": "chatcmpl-AonoLHzczRc9vHXpuohv76KsztCIX", "object": "chat.completion", "created": 1736671237, "model": "gpt-4o-mini-2024-07-18", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\n  \"MajorClaims\": [\n    {\n      \"ID\": \"MC1\",\n      \"Text\": \"dancing is an important part of culture\"\n    }\n  ],\n  \"Claims\": [\n    {\n      \"ID\": \"C1\",\n      \"Text\": \"dancing are significant part of culture that could show to something that people believe\"\n    },\n    {\n      \"ID\": \"C2\",\n      \"Text\": \"dancing can represent to civilization of that culture\"\n    },\n    {\n      \"ID\": \"C3\",\n      \"Text\": \"dancing is one of the ways people entertain themselves\"\n    }\n  ],\n  \"Premises\": [\n    {\n      \"ID\": \"P1\",\n      \"Text\": \"some cultur

### Batch 5

In [ ]:
# Upload der Batch Input-Datei auf die OpenAI-Plattform
batch_file_5 = upload_batch_file(BATCH_INPUT_PATH + "batch_input_5.jsonl", client)
batch_file_5

FileObject(id='file-BAg7ZagvUiaPgNUeoPjxVc', bytes=79286180, created_at=1736757463, filename='batch_input_5.jsonl', object='file', purpose='batch', status='processed', status_details=None)

In [ ]:
# Erstellen eines Batches
metadata_dict = {"description": "Batch 5/9 for Argument Mining"}
batch_5 = create_batch(batch_file_5.id, metadata_dict, client) # sofern die Batch Datei bereits hochgeladen wurde, aber nicht erfolgreich war, kann die Batch-ID erneut verwendet werden. Ein erneuter Uplaod ist nicht notwendig und würde der Datei eine neue ID zuweisen.
print(batch_5)

Batch(id='batch_6784d168bb288190bda2b11e3b42da3c', completion_window='24h', created_at=1736757608, endpoint='/v1/chat/completions', input_file_id='file-BAg7ZagvUiaPgNUeoPjxVc', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736844008, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'Batch 5/9 for Argument Mining'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


In [ ]:
# Status des Batches abfragen - Diese Funktion kann mehrfach aufgerufen werden, um den Status des Batches zu überprüfen, ohne Zusatzkosten zu verursachen.
batch_5_status = check_batch_status(batch_5.id, client)
print(batch_5_status)

Status: completed

Beschreibung des Batches: Batch 5/9 for Argument Mining
Anfragen gesamt: 323
Davon erfolgreich: 323
Davon fehlerhaft: 0
Erfolgreiche Abfragen können abgerufen werden mit ID: file-XCidZa5LAvqwrL52vCSaS2
Keine fehlerhaften Abfragen zum herunterladen vorhanden.

Batch(id='batch_6784d168bb288190bda2b11e3b42da3c', completion_window='24h', created_at=1736757608, endpoint='/v1/chat/completions', input_file_id='file-BAg7ZagvUiaPgNUeoPjxVc', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1736791620, error_file_id=None, errors=None, expired_at=None, expires_at=1736844008, failed_at=None, finalizing_at=1736791592, in_progress_at=1736757613, metadata={'description': 'Batch 5/9 for Argument Mining'}, output_file_id='file-XCidZa5LAvqwrL52vCSaS2', request_counts=BatchRequestCounts(completed=323, failed=0, total=323))


In [ ]:
# Abrufen und Speichern der Batch-Ergebnisse (Output-Datei)
batch_5_results = retrieve_and_save_batch_results(batch_5_status.output_file_id, BATCH_OUTPUT_PATH, "output-batch-5", client)
print(batch_5_results[:1000])

{"id": "batch_req_6785562970d48190890e1b3db0550e4e", "custom_id": "few-shot-40_essay081.txt", "response": {"status_code": 200, "request_id": "961d841d6ad27aec5dccc2ac0ef053e5", "body": {"id": "chatcmpl-ApJ1kDVOSD1DoKECkc3fctxXCi9xt", "object": "chat.completion", "created": 1736791232, "model": "gpt-4o-mini-2024-07-18", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\n  \"MajorClaims\": [\n    {\n      \"ID\": \"MC1\",\n      \"Text\": \"artists must be given freedom so that they will produce some really marvelous masterpiece\"\n    },\n    {\n      \"ID\": \"MC2\",\n      \"Text\": \"there should not be any restrictions on artists' work\"\n    }\n  ],\n  \"Claims\": [\n    {\n      \"ID\": \"C1\",\n      \"Text\": \"if there is control over artists' ideas, they will definitely lose their sense of creativity in the long run\"\n    },\n    {\n      \"ID\": \"C2\",\n      \"Text\": \"it is every human's right to be able to voice out their opinions in any ways as lo

### Batch 6

In [ ]:
# Upload der Batch Input-Datei auf die OpenAI-Plattform
batch_file_6 = upload_batch_file(BATCH_INPUT_PATH + "batch_input_6.jsonl", client)
batch_file_6

FileObject(id='file-58BfmzR7AzjHXBQKiRdeTG', bytes=79088105, created_at=1736788702, filename='batch_input_6.jsonl', object='file', purpose='batch', status='processed', status_details=None)

In [ ]:
# Erstellen eines Batches
metadata_dict = {"description": "Batch 6/9 for Argument Mining"}
batch_6 = create_batch(batch_file_6.id, metadata_dict, client) # sofern die Batch Datei bereits hochgeladen wurde, aber nicht erfolgreich war, kann die Batch-ID erneut verwendet werden. Ein erneuter Uplaod ist nicht notwendig und würde der Datei eine neue ID zuweisen.
print(batch_6)

Batch(id='batch_67860a74a03481909107e2a89f10a187', completion_window='24h', created_at=1736837748, endpoint='/v1/chat/completions', input_file_id='file-58BfmzR7AzjHXBQKiRdeTG', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1736924148, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'Batch 6/9 for Argument Mining'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


In [ ]:
# Status des Batches abfragen - Diese Funktion kann mehrfach aufgerufen werden, um den Status des Batches zu überprüfen, ohne Zusatzkosten zu verursachen.
batch_6_status = check_batch_status(batch_6.id, client)
print(batch_6_status)

Status: completed

Beschreibung des Batches: Batch 6/9 for Argument Mining
Anfragen gesamt: 322
Davon erfolgreich: 322
Davon fehlerhaft: 0
Erfolgreiche Abfragen können abgerufen werden mit ID: file-11jo846QQ1pmGXxp5JDNyZ
Keine fehlerhaften Abfragen zum herunterladen vorhanden.

Batch(id='batch_67860a74a03481909107e2a89f10a187', completion_window='24h', created_at=1736837748, endpoint='/v1/chat/completions', input_file_id='file-58BfmzR7AzjHXBQKiRdeTG', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1736839559, error_file_id=None, errors=None, expired_at=None, expires_at=1736924148, failed_at=None, finalizing_at=1736839526, in_progress_at=1736837752, metadata={'description': 'Batch 6/9 for Argument Mining'}, output_file_id='file-11jo846QQ1pmGXxp5JDNyZ', request_counts=BatchRequestCounts(completed=322, failed=0, total=322))


In [ ]:
# Abrufen und Speichern der Batch-Ergebnisse (Output-Datei)
batch_6_results = retrieve_and_save_batch_results(batch_6_status.output_file_id, BATCH_OUTPUT_PATH, "output-batch-6", client)
print(batch_6_results[:1000])

{"id": "batch_req_6786116710148190b712ba1d6fd42154", "custom_id": "few-shot-40-persona_essay037.txt", "response": {"status_code": 200, "request_id": "8dddee82cd827ecaaa9339f9b31859c8", "body": {"id": "chatcmpl-ApV8ul7jazbBrLlmszvK1SEYlAgly", "object": "chat.completion", "created": 1736837804, "model": "gpt-4o-mini-2024-07-18", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\n  \"MajorClaims\": [\n    {\n      \"ID\": \"MC1\",\n      \"Text\": \"international sporting occasions are essential in easing international tensions\"\n    },\n    {\n      \"ID\": \"MC2\",\n      \"Text\": \"International sporting events will make the world more peaceful\"\n    }\n  ],\n  \"Claims\": [\n    {\n      \"ID\": \"C1\",\n      \"Text\": \"international sporting events are a good change to create a multi-nation community of fans having the same passion\"\n    },\n    {\n      \"ID\": \"C2\",\n      \"Text\": \"people around the world understand each other more\"\n    },\n    {\

### Batch 7

In [ ]:
# Upload der Batch Input-Datei auf die OpenAI-Plattform
batch_file_7 = upload_batch_file(BATCH_INPUT_PATH + "batch_input_7.jsonl", client)
batch_file_7

FileObject(id='file-FVN2b1jfp2RS5JKC9ng7Em', bytes=79095032, created_at=1736924736, filename='batch_input_7.jsonl', object='file', purpose='batch', status='processed', status_details=None)

In [ ]:
# Erstellen eines Batches
metadata_dict = {"description": "Batch 7/9 for Argument Mining"}
batch_7 = create_batch(batch_file_7.id, metadata_dict, client) # sofern die Batch Datei bereits hochgeladen wurde, aber nicht erfolgreich war, kann die Batch-ID erneut verwendet werden. Ein erneuter Uplaod ist nicht notwendig und würde der Datei eine neue ID zuweisen.
print(batch_7)

Batch(id='batch_67875e439c0081909378f77faa6010ce', completion_window='24h', created_at=1736924739, endpoint='/v1/chat/completions', input_file_id='file-FVN2b1jfp2RS5JKC9ng7Em', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1737011139, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'Batch 7/9 for Argument Mining'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


In [ ]:
# Status des Batches abfragen - Diese Funktion kann mehrfach aufgerufen werden, um den Status des Batches zu überprüfen, ohne Zusatzkosten zu verursachen.
batch_7_status = check_batch_status(batch_7.id, client)
print(batch_7_status)

Status: completed

Beschreibung des Batches: Batch 7/9 for Argument Mining
Anfragen gesamt: 320
Davon erfolgreich: 320
Davon fehlerhaft: 0
Erfolgreiche Abfragen können abgerufen werden mit ID: file-SntzBR9MciREe9ZepXmZ6o
Keine fehlerhaften Abfragen zum herunterladen vorhanden.

Batch(id='batch_67875e439c0081909378f77faa6010ce', completion_window='24h', created_at=1736924739, endpoint='/v1/chat/completions', input_file_id='file-FVN2b1jfp2RS5JKC9ng7Em', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1736925178, error_file_id=None, errors=None, expired_at=None, expires_at=1737011139, failed_at=None, finalizing_at=1736925160, in_progress_at=1736924742, metadata={'description': 'Batch 7/9 for Argument Mining'}, output_file_id='file-SntzBR9MciREe9ZepXmZ6o', request_counts=BatchRequestCounts(completed=320, failed=0, total=320))


In [ ]:
# Abrufen und Speichern der Batch-Ergebnisse (Output-Datei)
batch_7_results = retrieve_and_save_batch_results(batch_7_status.output_file_id, BATCH_OUTPUT_PATH, "output-batch-7", client)
print(batch_7_results[:1000])

{"id": "batch_req_67875fe848308190b163df1129df2b14", "custom_id": "few-shot-40-persona_essay395.txt", "response": {"status_code": 200, "request_id": "e075021751db2fedcaf1daf7e6408d10", "body": {"id": "chatcmpl-AprlxQdnDoYgdeBHR0ieSwayx0UsH", "object": "chat.completion", "created": 1736924793, "model": "gpt-4o-mini-2024-07-18", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\n  \"MajorClaims\": [\n    {\n      \"ID\": \"MC1\",\n      \"Text\": \"these taxes are absolutely essential\"\n    },\n    {\n      \"ID\": \"MC2\",\n      \"Text\": \"taxes paying for state schools are necessary to be compulsory for all members of society no matter where their children enroll in\"\n    }\n  ],\n  \"Claims\": [\n    {\n      \"ID\": \"C1\",\n      \"Text\": \"affluent people effectively contribute to narrowing down the gap between rich and poor\"\n    },\n    {\n      \"ID\": \"C2\",\n      \"Text\": \"the tax reduction for parents of children studying in private schools wou

### Batch 8

In [ ]:
# Upload der Batch Input-Datei auf die OpenAI-Plattform
batch_file_8 = upload_batch_file(BATCH_INPUT_PATH + "batch_input_8.jsonl", client)
batch_file_8

FileObject(id='file-W8TyyDEwHrXiPGFpwGjVLg', bytes=79137498, created_at=1737010757, filename='batch_input_8.jsonl', object='file', purpose='batch', status='processed', status_details=None)

In [ ]:
# Erstellen eines Batches
metadata_dict = {"description": "Batch 8/9 for Argument Mining"}
batch_8 = create_batch(batch_file_8.id, metadata_dict, client) # sofern die Batch Datei bereits hochgeladen wurde, aber nicht erfolgreich war, kann die Batch-ID erneut verwendet werden. Ein erneuter Uplaod ist nicht notwendig und würde der Datei eine neue ID zuweisen.
print(batch_8)

Batch(id='batch_6788ae8069248190807e7a0ce2a5ebc9', completion_window='24h', created_at=1737010816, endpoint='/v1/chat/completions', input_file_id='file-W8TyyDEwHrXiPGFpwGjVLg', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1737097216, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'Batch 8/9 for Argument Mining'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


In [ ]:
# Status des Batches abfragen - Diese Funktion kann mehrfach aufgerufen werden, um den Status des Batches zu überprüfen, ohne Zusatzkosten zu verursachen.
batch_8_status = check_batch_status(batch_8.id, client)
print(batch_8_status)

Status: completed

Beschreibung des Batches: Batch 8/9 for Argument Mining
Anfragen gesamt: 320
Davon erfolgreich: 320
Davon fehlerhaft: 0
Erfolgreiche Abfragen können abgerufen werden mit ID: file-5jM9kQcHcNXSxJypakw1TC
Keine fehlerhaften Abfragen zum herunterladen vorhanden.

Batch(id='batch_6788ae8069248190807e7a0ce2a5ebc9', completion_window='24h', created_at=1737010816, endpoint='/v1/chat/completions', input_file_id='file-W8TyyDEwHrXiPGFpwGjVLg', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1737012493, error_file_id=None, errors=None, expired_at=None, expires_at=1737097216, failed_at=None, finalizing_at=1737012470, in_progress_at=1737010819, metadata={'description': 'Batch 8/9 for Argument Mining'}, output_file_id='file-5jM9kQcHcNXSxJypakw1TC', request_counts=BatchRequestCounts(completed=320, failed=0, total=320))


In [ ]:
# Abrufen und Speichern der Batch-Ergebnisse (Output-Datei)
batch_8_results = retrieve_and_save_batch_results(batch_8_status.output_file_id, BATCH_OUTPUT_PATH, "output-batch-8", client)
print(batch_8_results[:1000])

{"id": "batch_req_6788b4f714d081908df4f1d16c553782", "custom_id": "few-shot-40-cot_essay350.txt", "response": {"status_code": 200, "request_id": "cad64615ec7703d4819bcffc1e9412ed", "body": {"id": "chatcmpl-AqE9ifGMKq4tePxnXoXTf4ZFcSXZK", "object": "chat.completion", "created": 1737010834, "model": "gpt-4o-mini-2024-07-18", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\n  \"MajorClaims\": [\n    {\n      \"ID\": \"MC1\",\n      \"Text\": \"newspapers and magazines will be history with the time\"\n    },\n    {\n      \"ID\": \"MC2\",\n      \"Text\": \"digital media will rule over paper and magazines\"\n    }\n  ],\n  \"Claims\": [\n    {\n      \"ID\": \"C1\",\n      \"Text\": \"traditional paper media will come to an end very soon due to the increased usage of internet based media\"\n    },\n    {\n      \"ID\": \"C2\",\n      \"Text\": \"newspapers and magazines can not respond as fast as digital media\"\n    },\n    {\n      \"ID\": \"C3\",\n      \"Text\":

### Batch 9

In [ ]:
# Upload der Batch Input-Datei auf die OpenAI-Plattform
batch_file_9 = upload_batch_file(BATCH_INPUT_PATH + "batch_input_9.jsonl", client)
batch_file_9

FileObject(id='file-NUfgkd9nNh7XB4D4BN1isN', bytes=22248980, created_at=1737093443, filename='batch_input_9.jsonl', object='file', purpose='batch', status='processed', status_details=None)

In [ ]:
# Erstellen eines Batches
metadata_dict = {"description": "Batch 9/9 for Argument Mining"}
batch_9 = create_batch(batch_file_9.id, metadata_dict, client) # sofern die Batch Datei bereits hochgeladen wurde, aber nicht erfolgreich war, kann die Batch-ID erneut verwendet werden. Ein erneuter Uplaod ist nicht notwendig und würde der Datei eine neue ID zuweisen.
print(batch_9)

Batch(id='batch_6789f1518d488190addaf826cf4151c3', completion_window='24h', created_at=1737093457, endpoint='/v1/chat/completions', input_file_id='file-NUfgkd9nNh7XB4D4BN1isN', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1737179857, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'Batch 9/9 for Argument Mining'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


In [ ]:
# Status des Batches abfragen - Diese Funktion kann mehrfach aufgerufen werden, um den Status des Batches zu überprüfen, ohne Zusatzkosten zu verursachen.
batch_9_status = check_batch_status(batch_9.id, client)
print(batch_9_status)

Status: completed

Beschreibung des Batches: Batch 9/9 for Argument Mining
Anfragen gesamt: 90
Davon erfolgreich: 90
Davon fehlerhaft: 0
Erfolgreiche Abfragen können abgerufen werden mit ID: file-HqqCeMenJSYDTFvuStyVzx
Keine fehlerhaften Abfragen zum herunterladen vorhanden.

Batch(id='batch_6789f1518d488190addaf826cf4151c3', completion_window='24h', created_at=1737093457, endpoint='/v1/chat/completions', input_file_id='file-NUfgkd9nNh7XB4D4BN1isN', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1737094502, error_file_id=None, errors=None, expired_at=None, expires_at=1737179857, failed_at=None, finalizing_at=1737094496, in_progress_at=1737093458, metadata={'description': 'Batch 9/9 for Argument Mining'}, output_file_id='file-HqqCeMenJSYDTFvuStyVzx', request_counts=BatchRequestCounts(completed=90, failed=0, total=90))


In [ ]:
# Abrufen und Speichern der Batch-Ergebnisse (Output-Datei)
batch_9_results = retrieve_and_save_batch_results(batch_9_status.output_file_id, BATCH_OUTPUT_PATH, "output-batch-9", client)
print(batch_9_results[:1000])

{"id": "batch_req_6789f560ad1c81909d701102b78243ca", "custom_id": "few-shot-40-persona-cot_essay302.txt", "response": {"status_code": 200, "request_id": "60160904162abffef33aa4eea2d5a56b", "body": {"id": "chatcmpl-AqZeQvxI1L70irTZglv0gB1ygmFBj", "object": "chat.completion", "created": 1737093462, "model": "gpt-4o-mini-2024-07-18", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\n  \"MajorClaims\": [\n    {\n      \"ID\": \"MC1\",\n      \"Text\": \"saving some part of your earnings is essential\"\n    },\n    {\n      \"ID\": \"MC2\",\n      \"Text\": \"there is no other choice than saving money for some time\"\n    }\n  ],\n  \"Claims\": [\n    {\n      \"ID\": \"C1\",\n      \"Text\": \"you can not predict future developments, neither in your professional nor your personal life\"\n    },\n    {\n      \"ID\": \"C2\",\n      \"Text\": \"there are several uncertainties about your professional future\"\n    },\n    {\n      \"ID\": \"C3\",\n      \"Text\": \"you 

## Übersicht der Batches für Client

In [54]:
# Auflistung vergangener Batches
batches_data = client.batches.list(limit=11).data
for batch in batches_data:
    print(f"Metadata: {batch.metadata['description']}")
    print(f"Status: {batch.status}")
    print(f"Batch-ID: {batch.id}")
    if batch.status == "failed":
        print(f"Errors: {batch.errors}")
    print(f"-" * 40) 

# Info: Es scheint bisher keine möglichkeit zu geben Batches aus der Liste zu löschen. Das geht unter anderem aus folgenden Quellen hervor:
# - https://platform.openai.com/docs/guides/batch/batch-api#6-cancelling-a-batch
# - https://community.openai.com/t/how-to-restart-failed-batch-jobs-or-delete-batch-jobs/876352
# Die Test-Batches oder fehlgeschlagenen Batches können daher nicht aus der Liste entfernt werden.

Metadata: DAS IST EIN TEST
Status: completed
Batch-ID: batch_67a62b3adf9881909cfbb4033f3b08de
----------------------------------------
Metadata: Batch 9/9 for Argument Mining
Status: completed
Batch-ID: batch_6789f1518d488190addaf826cf4151c3
----------------------------------------
Metadata: Batch 8/9 for Argument Mining
Status: completed
Batch-ID: batch_6788ae8069248190807e7a0ce2a5ebc9
----------------------------------------
Metadata: Batch 7/9 for Argument Mining
Status: completed
Batch-ID: batch_67875e439c0081909378f77faa6010ce
----------------------------------------
Metadata: Batch 6/9 for Argument Mining
Status: completed
Batch-ID: batch_67860a74a03481909107e2a89f10a187
----------------------------------------
Metadata: Batch 6/9 for Argument Mining
Status: failed
Batch-ID: batch_67854ae1f8e08190bfbb2b6126ce4f2e
Errors: Errors(data=[BatchError(code='token_limit_exceeded', line=None, message='Enqueued token limit reached for gpt-4o-mini in organization org-1gP3awMqey1RnJpTBBhMoMP